- Ordem de afazeres

    - Construção do dataframe por completo (falta programar as adjacências) e aplicar o K-means

    - Complexidade do problema

    - Formas de resolver

    - Outros algoritmos que resolvem o problema

    - Algoritmos verificadores de resolução

    - Modelagem do problema NP completo para coloração de grafos

    - Abordagem pretendida para o problema

    - Tipos diferentes de sudoku

#### Sobre o problema

- O Sudoku Clássico consiste em uma grade 9x9 `(com um pequeno quadrado (célula) para cada número e uma outra divisão em grades de 3x3, representando as regiões (quadrantes))`, parcialmente preenchida, onde devem ser encontrados os locais corretos para preencher com números de 1 a 9, de modo com que nenhum número se repita na mesma linha, coluna ou quadrante.

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/Sudoku-by-L2G-20050714.svg/250px-Sudoku-by-L2G-20050714.svg.png" alt="Sudoku Clássico">

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/Sudoku-by-L2G-20050714_solution.svg/250px-Sudoku-by-L2G-20050714_solution.svg.png" alt="Sudoku Clássico Resolvido">
</center>

- Em algumas instâncias do sudoku parcialmente preenchido, é possível encontrar mais de uma solução possível, mas como o sudoku é considerado um problema de lógica pura, é esperado que exista somente uma solução possível. 

#### Variantes do sudoku

- O sudoku de 9x9 é o mais conhecido, porém, existem diversas variações do problema com outros tamanhos e abordagens. Abaixo segue uma representação de um sudoku 9x9, mas com seus quadrantes definidos aleatoriamente.

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Nonomino_Sudoku.svg/220px-Nonomino_Sudoku.svg.png" alt="Sudoku Variante">
</center>

- Modelagem do sudoku de tamanho 10x10

<center>
<img src="10x10.png" alt="Sudoku 10x10" width="250" height="250">
</center>

#### Abordagem planejada

- Planejamos uma abordagem `não-supervisionada` de máquina, onde vamos modelar a `grade` do sudoku para um formato de grafo. O tamanho da grade do sudoku não vai ser fixo, estamos planejando trabalhar com um `n-sudoku`. Para representar as restrições do sudoku, planejamos utilizar algum algoritmo de coloração de vértices `(em aberto ainda)` e considerar cada cluster como sendo uma cor. Exemplo, para um sudoku 9x9, teríamos 81 vértices e 9 clusters, onde cada vértice é adjacente a todos os outros vértices de sua linha, coluna e região. 

#### Problemas esperados

- Para uma abordagem com n-sudokus, é esperado que para alguns tamanhos de grade, como 10x10 ou 11x11, a forma como os quadrantes vão ser construidos pode acabar se tornando um contra-tempo na hora de visualizar melhor o agrupamento dos clusters. Por isso, optamos em trabalhar com grades que sejam múltiplas de três, de forma a padronizar as saídas e melhorar a visualização.

- Segundo problema esperado, devido ao número elevado de ligações e de adjacências entre os vértices, é provável que acabe ocorrendo colisões de dados ao executar o K-means. Estamos pensando em uma abordagem que consiga afastar os dados o máximo possível uns dos outros.

<center>
<img src="ligações.png" alt="Sudoku 10x10">
</center>

#### Bibliotecas

In [ ]:
import pandas as pd
import numpy as np

#### Criação do dataframe

In [ ]:
def create_sudoku_dataframe(N):

    data = []

    for line in range(1, N + 1): # For the lines
        for column in range(1, N + 1): # For de columns
            quadrant = (line - 1) // int(np.sqrt(N)) * int(np.sqrt(N)) + (column - 1) // int(np.sqrt(N)) + 1
                # Formula for calculating quadrants in Sudoku
            data.append([line, column, quadrant, None, None, None])

    columns = ['line_v', 'column_v', 'quadrant_v', 'adjacent_vertices_line', 'adjacent_vertices_column', 'adjacent_vertices_quadrant']
    data_frame = pd.DataFrame(data, columns = columns)

    return data_frame.values

N = 9

if N % 3 == 0: # It will only accept sudoku sizes that are multiples of 3 to standardize the quadrants
    sudoku_df = create_sudoku_dataframe(N)
    print(sudoku_df)
else:
    print("The size of N is not a multiple of 3.")
